In [1]:
import numpy as np
import pandas as pd

import os
#os.sys.path
import sys
sys.path.append('../src')

from pathlib import Path

from PIL import Image
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as imgplt
import code1_data as go

In [2]:
inp1_path = Path.home()/'Iron'/'inp1'

train_path = inp1_path/'train'

testvalidation_path = inp1_path/'test_validation'

fer_csv = inp1_path/'Fer.csv'

train_csv = train_path/'Training_Data.csv'

test_csv = testvalidation_path/'Testing_Data.csv'

validation_csv = testvalidation_path/'Validation_Data.csv'


In [3]:
df_fer = pd.read_csv(fer_csv,encoding = "ISO-8859-1")
df_train = pd.read_csv(train_csv,encoding = "ISO-8859-1")
df_test = pd.read_csv(test_csv,encoding = "ISO-8859-1")
df_val = pd.read_csv(validation_csv,encoding = "ISO-8859-1")

In [4]:
emotions = ('angry','disgust','fear','happy','sad','surprise','neutral')

## fer2013

In [5]:
df = df_fer.copy( )

In [8]:
print(df.shape, df.size, df.columns)

(35887, 3) 107661 Index(['emotion', 'pixels', 'Usage'], dtype='object')


In [9]:
image_size=(48,48)
pixels = df['pixels'].tolist() # Converting the relevant column element into a list for each row
width, height = 48, 48
faces = []

for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')] # Splitting the string by space character as a list
    face = np.asarray(face).reshape(width, height) #converting the list to numpy array in size of 48*48
    face = cv2.resize(face.astype('uint8'),image_size) #resize the image to have 48 cols (width) and 48 rows (height)
    faces.append(face.astype('float32')) #makes the list of each images of 48*48 and their pixels in numpyarray form

faces = np.asarray(faces) #converting the list into numpy array
faces = np.expand_dims(faces, -1) #Expand the shape of an array -1=last dimension => means color space
emotions = pd.get_dummies(df['emotion']).to_numpy() #doing the one hot encoding type on emotions

In [10]:
print(faces[0].shape)
print(faces.shape)

(48, 48, 1)
(35887, 48, 48, 1)


### Standardize or Normalize?

In [11]:
# normalize (from 0 to 1)
x = faces.astype('float32')
x = x / 255.0

# scale pixel value (from -1 to 1)
x = x - 0.5
x = x * 2.0

In [12]:
emotions[0]

array([1, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [14]:
q,t = emotions.shape
q = len(x)
q_train = int((0.8) * q)

## train

In [15]:
# Training data
train_x = x[:q_train]
train_y = emotions[:q_train]

# Validation data
val_x = x[q_train:]
val_y = emotions[q_train:]

train_data = (train_x, train_y)
val_data = (val_x, val_y)

In [18]:
print("\n")
print('train_x.shape (pixels): ',train_x.shape)  # ==> 4 dims -  no of images , width , height , color
print("\n")
print('train_y.shape: (labels): ',train_y.shape)
print("\n")
print('val_x.shape (pixels): ',val_x.shape)
print("\n")
print('val_y.shape: (labels): ',val_y.shape)
print("\n")



train_x.shape (pixels):  (28709, 48, 48, 1)


train_y.shape: (labels):  (28709, 7)


val_x.shape (pixels):  (7178, 48, 48, 1)


val_y.shape: (labels):  (7178, 7)




In [20]:
train_x[0]

array([[[-0.45098037],
        [-0.372549  ],
        [-0.35686272],
        ...,
        [-0.5921569 ],
        [-0.6627451 ],
        [-0.6784314 ]],

       [[-0.49019605],
        [-0.52156866],
        [-0.54509807],
        ...,
        [-0.56078434],
        [-0.5921569 ],
        [-0.654902  ]],

       [[-0.60784316],
        [-0.6627451 ],
        [-0.5764706 ],
        ...,
        [-0.6156863 ],
        [-0.56078434],
        [-0.6313726 ]],

       ...,

       [[-0.2862745 ],
        [-0.49019605],
        [-0.67058825],
        ...,
        [-0.4352941 ],
        [-0.56078434],
        [-0.6627451 ]],

       [[-0.3960784 ],
        [-0.35686272],
        [-0.38039213],
        ...,
        [-0.17647058],
        [-0.45098037],
        [-0.6392157 ]],

       [[-0.3960784 ],
        [-0.4352941 ],
        [-0.34117645],
        ...,
        [-0.16862744],
        [-0.14509803],
        [-0.35686272]]], dtype=float32)

## predict(validation)

## test